In [1]:
#Importação de Bibliotecas
import pandas as pd
import requests,json
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [2]:
#Função para criar Token de acesso de um usuário em específico: create_user_access_token()

def create_user_access_token(username,clientID,secret):
    '''Função que utiliza a biblioteca Spotipy para realizar a criação de
       um token específico para coleta de dados pessoais do usuário
    '''
    # Configurações do aplicativo Spotify
    CLIENT_ID = clientID
    CLIENT_SECRET = secret
    REDIRECT_URI = 'http://localhost:8888/callback' 
    USERNAME = username  

    # Escopo de permissões que você deseja obter
    SCOPE = 'user-read-recently-played user-read-private user-read-email user-top-read'  # Solicita acesso ao histórico de reprodução recente do usuário

    # Autorização do usuário
    token = util.prompt_for_user_token(username=USERNAME,
                                       scope=SCOPE,
                                       client_id=CLIENT_ID,
                                       client_secret=CLIENT_SECRET,
                                       redirect_uri=REDIRECT_URI)

    return 'Bearer '+token

In [3]:
#Função para coletar os dados mais recentes de execução

def query_recent(token):
    '''Função que realiza a consulta dos últimos registros do player do spotify.
       Esta função executa a create_user_access_token(), pois necessita de um escopo específico.
       O limite padrão é 150, mas pode ter como entrada outro valor.
    '''
    headers = {
        "Authorization":  token
            }

    # Definir URL para pesquisa
    recently_played_url = "https://api.spotify.com/v1/me/player/recently-played"

    # Definindo parâmetro de limite de resultado. Inicialmente definido para 50. Usar com parcimônia, pois o Spotify bloqueia consultas muito longas
    params = {
        "limit": 50
    }

    # Realizar o Request na API do Spotify
    response = requests.get(recently_played_url, headers=headers, params=params)
    recently_played_data = response.json()

    # Extrair dados da resposta
    tracks = []
    for item in recently_played_data.get("items", []):
        track = item.get("track")
        if track:
            track_info = {
                "Nome da Música": track.get("name"),
                "Nome do Artista": track.get("artists")[0].get("name"),
                "Nome do Álbum": track.get("album").get("name"),
                "Link da Música": track.get("uri"),
                "Data de Execução": item.get("played_at")
            }
            tracks.append(track_info)
    #Retorno da Função
    return tracks

In [4]:
#Criação de Token do Spotify.
#O Token tem validade de 60 minutos
#Username pode ser coletado diretamente do painel de gerenciamento da conta do usuário
#O Client ID e o Secret precisam ser criadas no Dashboard de API do Spotify
token = create_user_access_token('22rxsto4zfjdqqyfdhx27v3jq','7d486b31ccdc419798ea333e9aa9c91c','018d8dfeb158496b80452b6a4ccbe28d')

In [8]:
#Criando Dataframe com o resultado do request à API do Spotify
ultimas_musicas = query_recent(token)
dataframe_ultimas_musicas = pd.DataFrame.from_dict(ultimas_musicas)
dataframe_ultimas_musicas
with open('arquivo.json', 'w') as arquivo:
    json.dump(ultimas_musicas, arquivo)


Nome da Música Nome do Artista  \
0                   Coisas Simples            YOÙN   
1                             AWAY          Oxlade   
2                      No Dey Form      Babyboy AV   
3                       Terminator           Asake   
4                        Calm Down            Rema   
5                           Honest         SPINALL   
6                           Koroba     Tiwa Savage   
7            In Fact (feat. Gabzy)        melvitto   
8                    Love Language            Juls   
9                             Muse           Asake   
10                          Mayana             Aṣa   
11                       Testimony            Bnxn   
12                      Born Again   Adekunle Gold   
13                           Fever          Soundz   
14                       Yebo/Sema          Masego   
15                   Love Somebody          Rotimi   
16                         Segredu   Mayra Andrade   
17                    pedra do sal            BIAB   
18                            Jazz            YOÙN   
19                         Faminta    Bia Ferreira   
20       Coffee (Don't Read Signs)           Odeal   
21              Fim das Tentativas    Rico Dalasam   
22                             Luv           Melly   
23                           LadoB      Afrocidade   
24                    We Wan Comot     Kizz Daniel   
25                         Be Easy           Odeal   
26                           Facin            UANA   
27          Bota Pagodão Ponto Net     Rachel Reis   
28                           Atele          Lasmid   
29                           Omoge      Steph Ocho   
30                  Carta ao Tempo            BIAB   
31                     Pray For Me            Ayox   
32                     (In)Verdade           Melly   
33                    Boys Are Bad     Kizz Daniel   
34               Amor Sem Barreira     Rachel Reis   
35                         Collabo    June Freedom   
36  Very Connected (feat. Flavour)          Fiokee   
37                        Tomorrow       M.anifest   
38                           Ceder           Melly   
39       Me Peguei Pensando em Nós            BIAB   
40                         Me Veja     Rachel Reis   
41                         Inebrio            YOÙN   
42                 Eu Tava Em Casa    Bia Ferreira   
43                           Usual        melvitto   
44                   Set Me Alight  Nelson Freitas   
45             Silver Tongue Devil          Masego   
46                             𝘠𝘚𝘓    June Freedom   
47                          Flerte     Rachel Reis   
48                        Oshofree             YKB   
49                             304      Afrocidade   

                     Nome do Álbum                        Link da Música  \
0                   Coisas Simples  spotify:track:4pvK5AbNctlQKAy2Fdvsmp   
1                             AWAY  spotify:track:1wfIqsTXmMVtZxYk8uWxGa   
2                      No Dey Form  spotify:track:1a5glQzge8OW5w2izckOGN   
3          Mr. Money With The Vibe  spotify:track:5ycsqezujKrwviU3VFhci1   
4                     Rave & Roses  spotify:track:3BnDvpeuGOj21Ir2aVEtQo   
5                          TOP BOY  spotify:track:2vnNYl3UDY9Grubs0VAK5m   
6                            Celia  spotify:track:18t2ofQiViidZHxOBVjpD1   
7                 THENIGHTISYOUNG.  spotify:track:2rFPUq8a7gvO0IuaN9LpV7   
8               Sounds of My World  spotify:track:7Jqa7kWKN4d2Fn7FESctLz   
9          Mr. Money With The Vibe  spotify:track:3Nx9nMHMFgkeF8xpVk1Ev0   
10                               V  spotify:track:5WeLc7gPjNabQ1AMk5lxrl   
11                       Testimony  spotify:track:5L845uQOeyOSSAhGPFEmLc   
12             Catch Me If You Can  spotify:track:2XhX2hzgu66MKw09tbbwCE   
13                           Fever  spotify:track:3GAu2fSg9deV207noxMKl0   
14  Studying Abroad: Extended Stay  spotify:track:0ed7g01Suk6ITt0vwT2BgD   
15         All or Nothing (Deluxe)  spotify:track